In [38]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error

import xgboost as xgb

import pickle

pd.set_option('display.max_columns', None)

In [3]:
from flask import Flask, render_template, request, redirect, url_for, flash
from flask_sqlalchemy import SQLAlchemy
import requests

In [4]:
#audio features import
audio_features = pd.read_csv('../data/audio_features.csv')

#spotify import
sp_artist_release = pd.read_csv('../data/sp_artist_release.csv')
sp_artist_track = pd.read_csv('../data/sp_artist_track.csv')
sp_artist = pd.read_csv('../data/sp_artist.csv')
sp_release = pd.read_csv('../data/sp_release.csv')
sp_track = pd.read_csv('../data/sp_track.csv')

In [5]:
df_all = audio_features.merge(sp_track, on='isrc', how='inner')
df_all = df_all.merge(sp_artist_track, on='track_id', how='inner')
df_all = df_all.drop(columns=['updated_on_x'])
df_all = df_all.merge(sp_artist, on='artist_id', how='inner')
df_all = df_all.merge(sp_release, on='release_id', how='inner')

df_all['duration_sec'] = df_all['duration_ms_x']/1000
df_all['duration_sec'] = df_all['duration_sec'].round(0).astype(int)
df_all = df_all.drop(columns=['duration_ms_x'])
df_all = df_all[['isrc', 'release_id', 'track_title', 'artist_name','release_title', 'album_type', 'release_date', 'acousticness',
                       'danceability', 'duration_sec', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness',
                       'mode', 'speechiness', 'tempo', 'time_signature', 'valence', 'explicit', 'popularity']]

df_all['key'] = df_all['key'].astype(str)
df_all['mode'] = df_all['mode'].astype(str)
df_all['time_signature'] = df_all['time_signature'].astype(str)
df_all = df_all.drop(columns=['time_signature'])

#df_all['release_year'] = df_all['release_date'].str.split('-').str[0].astype(int)
#df_all = df_all[df_all['release_year']>=2000]
df_all = df_all.drop(columns=['release_date'])

df_all = df_all[df_all['acousticness'].between(0.0, 0.2)]
df_all = df_all[df_all['danceability'].between(0.1, 1.0)]
df_all = df_all[df_all['duration_sec'].between(60, 600)]
df_all = df_all[df_all['energy'].between(0.0, 1.0)]
df_all = df_all[df_all['liveness'].between(0.0, 0.4)]
df_all = df_all[df_all['loudness'].between(-20.0, 0.0)]
df_all = df_all[df_all['speechiness'].between(0.02, 0.4)]
df_all = df_all[df_all['tempo'].between(70, 180)]
#df_all = df_all[df_all['tempo'].between(50, 200)]

pop_min = df_all['popularity'].min()
pop_max = df_all['popularity'].max()
df_all['popularity'] = round((df_all['popularity']-pop_min)/(pop_max-pop_min)*100, 0)

df_all = df_all[(df_all['duration_sec']>=60) & (df_all['duration_sec']<=600)]

#df_all['release_date'] = pd.to_datetime(df_all['release_date'], errors='coerce').dt.to_period('Y')

df_all = df_all.groupby('isrc').agg({
    'release_id': 'first',
    'track_title': 'first',
    'artist_name': 'first',
    'release_title': 'first',
    'album_type': 'first',
    #'release_year': 'first',
    'acousticness': 'first',
    'danceability': 'first',
    'duration_sec': 'first',
    'energy': 'first',
    'instrumentalness': 'first',
    #'key': 'first',
    'liveness': 'first',
    'loudness': 'first',
    #'mode': 'first',
    'speechiness': 'first',
    'tempo': 'first',
    #'time_signature': 'first',
    'valence': 'first',
    #'explicit': 'first',
    'popularity': 'mean'
}).reset_index()

df_all['popularity'] = df_all['popularity'].round(0).astype(int)

#register min and max values of all columns that are standardized later
#release year
#release_year_min = df_all['release_year'].min()
#release_year_max = df_all['release_year'].max()
#acousticness
acousticness_min = df_all['acousticness'].min()
acousticness_max = df_all['acousticness'].max()
#danceability
danceability_min = df_all['danceability'].min()
danceability_max = df_all['danceability'].max()
#duration_sec
duration_sec_min = df_all['duration_sec'].min()
duration_sec_max = df_all['duration_sec'].max()
#energy
energy_min = df_all['energy'].min()
energy_max = df_all['energy'].max()
#instrumentalness
instrumentalness_min = df_all['instrumentalness'].min()
instrumentalness_max = df_all['instrumentalness'].max()
#liveness
liveness_min = df_all['liveness'].min()
liveness_max = df_all['liveness'].max()
#loudness
loudness_min = df_all['loudness'].min()
loudness_max = df_all['loudness'].max()
#speechiness
speechiness_min = df_all['speechiness'].min()
speechiness_max = df_all['speechiness'].max()
#tempo
tempo_min = df_all['tempo'].min()
tempo_max = df_all['tempo'].max()
#valence
valence_min = df_all['valence'].min()
valence_max = df_all['valence'].max()
#popularity
popularity_min = df_all['popularity'].min()
popularity_max = df_all['popularity'].max()

# reduce lower popularity values to have a better distribution
max_rows_per_value = 50000

sampled_df = pd.DataFrame(columns=df_all.columns)

for value in sorted(df_all['popularity'].unique()):
    subset = df_all[df_all['popularity'] == value]
    
    if len(subset) > max_rows_per_value:
        sampled_subset = subset.sample(n=max_rows_per_value, random_state=42)
    else:
        sampled_subset = subset
    
    sampled_df = pd.concat([sampled_df, sampled_subset])

sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)
sampled_df.reset_index(drop=True, inplace=True)
df_all = sampled_df

df_all.to_csv('../data/df_all.csv', index=False)

/var/folders/5j/86vb95z109v6f0wkwc7b40240000gn/T/ipykernel_70599/3021875524.py:118: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sampled_df = pd.concat([sampled_df, sampled_subset])


In [7]:
df_all = pd.read_csv('../data/df_all.csv')

In [9]:
#print('release_year_min:', release_year_min)
#print('release_year_max:', release_year_max)
print('acousticness_min:', acousticness_min)
print('acousticness_max:', acousticness_max)
print('danceability_min:', danceability_min)
print('danceability_max:', danceability_max)
print('duration_sec_min:', duration_sec_min)
print('duration_sec_max:', duration_sec_max)
print('energy_min:', energy_min)
print('energy_max:', energy_max)
print('instrumentalness_min:', instrumentalness_min)
print('instrumentalness_max:', instrumentalness_max)
print('liveness_min:', liveness_min)
print('liveness_max:', liveness_max)
print('loudness_min:', loudness_min)
print('loudness_max:', loudness_max)
print('speechiness_min:', speechiness_min)
print('speechiness_max:', speechiness_max)
print('tempo_min:', tempo_min)
print('tempo_max:', tempo_max)
print('valence_min:', valence_min)
print('valence_max:', valence_max)
print('popularity_min:', popularity_min)
print('popularity_max:', popularity_max)

acousticness_min: 0.0
acousticness_max: 0.2
danceability_min: 0.1
danceability_max: 0.998
duration_sec_min: 60
duration_sec_max: 600
energy_min: 2.02e-05
energy_max: 1.0
instrumentalness_min: 0.0
instrumentalness_max: 1.0
liveness_min: 0.00381
liveness_max: 0.4
loudness_min: -20.0
loudness_max: 0.0
speechiness_min: 0.0217
speechiness_max: 0.4
tempo_min: 70
tempo_max: 180
valence_min: 0.0
valence_max: 1.0
popularity_min: 0
popularity_max: 100


In [10]:
columns_to_scale = [
                    #'release_year',
                    'acousticness',
                    'danceability',
                    'duration_sec',
                    'energy',
                    'instrumentalness',
                    'liveness',
                    'loudness',
                    'speechiness',
                    'tempo',
                    #'time_signature',
                    'valence'
                    ]

#STANDARD SCALER: scale numeric columns
# scaler = StandardScaler()

#MINMAX SCALER: scale numeric columns
scaler = MinMaxScaler()

pickle.dump(scaler, open('../models/minmaxscaler.sav', 'wb'))
joblib.dump(scaler, '../models/minmax_scaler.pkl')

df_all_scaled = df_all.copy()
df_all_scaled[columns_to_scale] = scaler.fit_transform(df_all_scaled[columns_to_scale])

In [108]:
X = df_all_scaled.drop(columns=['popularity'])
#object_columns = X.select_dtypes(include=['object']).columns
#X = pd.get_dummies(X, columns=object_columns)

y = df_all_scaled['popularity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_knn = X_train.copy()
X_train = X_train.drop(columns=['isrc', 'release_id', 'track_title', 'artist_name','release_title', 'album_type'])
X_test = X_test.drop(columns=['isrc', 'release_id', 'track_title', 'artist_name','release_title', 'album_type'])

X.to_csv('../data/X.csv', index=False)
X_train.to_csv('../data/X_train.csv', index=False)
X_train_knn.to_csv('../data/X_train_knn.csv', index=False)
y_train.to_csv('../data/y_train.csv', index=False)
X_test.to_csv('../data/X_test.csv', index=False)
y_test.to_csv('../data/y_test.csv', index=False)

In [109]:
X = pd.read_csv('../data/X.csv')
X_train = pd.read_csv('../data/X_train.csv')
X_train_knn = pd.read_csv('../data/X_train_knn.csv')
y_train = pd.read_csv('../data/y_train.csv')
X_test = pd.read_csv('../data/X_test.csv')
y_test = pd.read_csv('../data/y_test.csv')

In [74]:
regression_model_load = True

if regression_model_load:
    regression_model = pickle.load(open('../models/regression_model.pkl', 'rb'))

else:
    regression_model = LinearRegression()
    regression_model.fit(X_train, y_train)
    pickle.dump(regression_model, open('../models/regression_model.pkl', 'wb'))

y_pred = regression_model.predict(X_test)

pop_pred_min = y_pred.min()
pop_pred_max = y_pred.max()

y_pred = (y_pred-pop_pred_min)/(pop_pred_max-pop_pred_min)*100
y_pred = np.round(y_pred)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
rmse = math.sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")

Mean Squared Error: 1083.7863123464945
Root Mean Squared Error: 32.92090995623442


In [65]:
xgboost_model_load = True

label_encoder = LabelEncoder()
y_train_tt = label_encoder.fit_transform(y_train)

if xgboost_model_load:
    xgboost_model = pickle.load(open('../models/xgboost_model.pkl', 'rb'))

else:
    xgboost_model = xgb.XGBClassifier()
    xgboost_model.fit(X_train, y_train_tt)
    pickle.dump(xgboost_model, open('../models/xgboost_model.pkl', 'wb'))

y_pred_encoded = xgboost_model.predict(X_test)
y_pred_original = label_encoder.inverse_transform(y_pred_encoded)

pop_pred_min = y_pred_original.min()
pop_pred_max = y_pred_original.max()

y_pred_original = (y_pred_original-pop_pred_min)/(pop_pred_max-pop_pred_min)*100
y_pred_original = np.round(y_pred_original).astype(int)

mse = mean_squared_error(y_test, y_pred_original)
print(f"Mean Squared Error: {mse}")
rmse = math.sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")

/Users/finn/Desktop/MusicHype/musichype_env/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/finn/Desktop/MusicHype/musichype_env/lib/python3.9/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/finn/Desktop/MusicHype/musichype_env/lib/python3.9/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Users/finn/Desktop/MusicHype/musichype_env/lib/python3.9/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be 

Mean Squared Error: 288.4320501817348
Root Mean Squared Error: 16.98328737852995


In [87]:
# Create a new DataFrame for the new data you want to make predictions on
sample_data = {
    #'release_year': [0.995, 0.90, 0.99],
    'acousticness': [1.0, 0.5, 0.053313],
    'danceability': [1.0, 0.5, 0.763000],
    'duration_sec': [0.43, 0.14, 0.28],
    'energy': [1.0, 0.5, 0.628000],
    'instrumentalness': [1.0, 0.5, 0.000000],
    'liveness':[1.0, 0.5, 0.114000],
    'loudness': [1.0, 0.5, 0.826907],
    'speechiness': [1.0, 0.5, 0.051653],
    #'time_signature',
    'tempo': [1.0, 0.5, 0.564000],
    'valence': [1.0, 0.5, 0.193000]
}

columns_to_scale = [
                    #'release_year',
                    'acousticness',
                    'danceability',
                    'duration_sec',
                    'energy',
                    'instrumentalness',
                    'liveness',
                    'loudness',
                    'speechiness',
                    'tempo',
                    #'time_signature',
                    'valence'
                    ]

sample_df = pd.DataFrame(sample_data)

predictions = regression_model.predict(sample_df)

for i in predictions:
    i = i[0]
    pop_under = df_all[df_all['popularity']<=i].shape[0]
    pop_all = df_all.shape[0]
    pop_place = pop_under/pop_all*100
    pop_place = int(round(pop_place, 0))
    i = int(round(i, 0))
    print("Dein Song hat eine erwartete Popularität von %s%% und liegt über %s%% aller anderen Songs"%(i, pop_place))

Dein Song hat eine erwartete Popularität von 13% und liegt über 59% aller anderen Songs
Dein Song hat eine erwartete Popularität von 21% und liegt über 75% aller anderen Songs
Dein Song hat eine erwartete Popularität von 21% und liegt über 75% aller anderen Songs


In [ ]:
Dein Song hat eine erwartete Popularität von 13% und liegt über 59% aller anderen Songs
Dein Song hat eine erwartete Popularität von 21% und liegt über 75% aller anderen Songs
Dein Song hat eine erwartete Popularität von 21% und liegt über 75% aller anderen Songs

In [11]:
#scaler = joblib.load('../models/minmax_scaler.pkl')

In [102]:
pred_model = 'regression' # regression or xgboost

# Create a new DataFrame for the new data you want to make predictions on

sample_data = {
    #'release_year': [2000],
    'acousticness': [0.5],
    'danceability': [0.5],
    'duration_sec': [130],
    'energy': [0.5],
    'instrumentalness': [0.5],
    'liveness':[0.5],
    'loudness': [1],
    'speechiness': [0.5],
    #'time_signature',
    'tempo': [100],
    'valence': [0.5]
}

sample_data = {
    #'release_year': [2022],
    'acousticness': [0.00453],
    'danceability': [0.545],
    'duration_sec': [215],
    'energy': [0.641],
    'instrumentalness': [0.000066],
    'liveness':[0.171],
    'loudness': [-6.3],
    'speechiness': [0.0998],
    #'time_signature',
    'tempo': [122],
    'valence': [0.464]
}

sample_data = {
    #'release_year': [0.995, 0.90, 0.99],
    'acousticness': [1.0, 0.5, 0.153313],
    'danceability': [1.0, 0.5, 0.763000],
    'duration_sec': [0.43, 0.14, 0.28],
    'energy': [1.0, 0.5, 0.28000],
    'instrumentalness': [1.0, 0.5, 0.000000],
    'liveness':[1.0, 0.5, 0.114000],
    'loudness': [1.0, 0.5, 0.826907],
    'speechiness': [1.0, 0.5, 0.051653],
    #'time_signature',
    'tempo': [1.0, 0.5, 0.564000],
    'valence': [1.0, 0.5, 0.193000]
}

columns_to_scale = [
                    #'release_year',
                    'acousticness',
                    'danceability',
                    'duration_sec',
                    'energy',
                    'instrumentalness',
                    'liveness',
                    'loudness',
                    'speechiness',
                    'tempo',
                    #'time_signature',
                    'valence'
                    ]

sample_df = pd.DataFrame(sample_data)

df_all_scaled = df_all.copy()
df_all_scaled[columns_to_scale] = scaler.fit_transform(df_all_scaled[columns_to_scale])

sample_df_scaled = scaler.transform(sample_df)

if pred_model == 'regression':
    print('Predicting with regression model')
    predictions = regression_model.predict(sample_df_scaled)
elif pred_model == 'xgboost':
    print('Predicting with xgboost model')
    predictions = xgboost_model.predict(sample_df_scaled)

for i in predictions:
    if pred_model == 'regression':
        i = i[0]
    pop_under = df_all[df_all['popularity']<=i].shape[0]
    pop_all = df_all.shape[0]
    pop_place = pop_under/pop_all*100
    pop_place = int(round(pop_place, 0))
    i = int(round(i, 0))
    print("Dein Song hat eine erwartete Popularität von %s und liegt über %s%% aller anderen Songs"%(i, pop_place))

Predicting with regression model
Dein Song hat eine erwartete Popularität von 38 und liegt über 92% aller anderen Songs
Dein Song hat eine erwartete Popularität von 37 und liegt über 92% aller anderen Songs
Dein Song hat eine erwartete Popularität von 34 und liegt über 90% aller anderen Songs


/Users/finn/Desktop/MusicHype/musichype_env/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [13]:
# acousticness 0.0 - 0.2
# danceability 0.1 - 1.0
# duration_sec 60 - 600
# energy 0.0 - 1.0

# liveness 0.0 - 0.4
# loudness -20 - 0.0
# speechiness 0.02 - 0.4
# tempo 70 - 180

#plt.hist(df_all['valence'], bins=100)
#plt.show()

In [111]:
k = 3 # number of neighbors to find
nn = NearestNeighbors(n_neighbors=k)
nn.fit(X_train)

distances, neighbor_indices = nn.kneighbors(sample_df_scaled)

columns_sample_neighbors = ['isrc', 'track_title', 'artist_name','release_title']
df_neighbors_info = pd.DataFrame(columns=columns_sample_neighbors)

for i in list(X_train.iloc[neighbor_indices[0]].index):
    print(i)
    sample_neighbors = X_train.iloc[neighbor_indices[0]]
    sample_neighbors = sample_neighbors.reset_index()
    X_merger = X.reset_index()
    sample_neighbors = sample_neighbors.merge(X_merger, on='index', how='inner')
    sample_neighbors = sample_neighbors[['index', 'isrc', 'track_title', 'artist_name', 'release_title']]
    sample_neighbors = sample_neighbors.merge(sp_release, on='release_title', how='inner')
    print(X_train_knn.loc[i][['isrc', 'track_title', 'artist_name','release_title']])
    df_neighbors_info_append = X_train_knn.loc[i][['isrc', 'track_title', 'artist_name','release_title']]
    df_neighbors_info_append = pd.DataFrame(df_neighbors_info_append).T
    #df_neighbors_info = df_neighbors_info.append(X_train_knn.loc[i][['isrc', 'track_title', 'artist_name','release_title']], ignore_index=True)
    df_neighbors_info = pd.concat([df_neighbors_info, df_neighbors_info_append], axis=0)

344079
isrc             GBQXM0600070
track_title       That's Long
artist_name         Dubbledge
release_title    Fist of Jah!
Name: 344079, dtype: object
496529
isrc              CA5KR0024530
track_title      Oye Loco 2020
artist_name         Nico Parga
release_title      Guarachaton
Name: 496529, dtype: object
126687
isrc                   USKO10800501
track_title        That's A Soldier
artist_name             Sheek Louch
release_title    Silverback Gorilla
Name: 126687, dtype: object
